#### Load Modules

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### Cross reference Wake P_Ids with other data types

In [5]:
#1. Load Wake Data and work out which P_Id's are not there in other sleep data types. 
#These need to be dropped --> p_id_to_drop
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'

p_id_to_drop = []

paths = joblib.load(folder + 'Wake' + '_paths.pkl')
wake_groups = pd.Series(paths['s_p_id'])
wake_groups_counts = wake_groups.value_counts()

for ind, val in zip(wake_groups_counts.index, wake_groups_counts):
    #loop through the groups of each data type
    # print('Outer index is: ' + str(ind) ) for debugging
    for data_type in ['N2', 'N3','REM', 'N1']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
 
        if ind in groups.values: 
            pass 
        else:
            #print(ind)
            p_id_to_drop.append(ind)
            wake_groups_counts = wake_groups_counts.drop(index = ind)
            break

print(p_id_to_drop)

#2. Work Out which P_id's are present only once in other data types but twice in Wake 
#These need to be semi dropped --> single_pids

wake_multiple_pids = wake_groups_counts[wake_groups_counts > 1]

single_pids = []

for ind, val in zip(wake_multiple_pids.index, wake_multiple_pids):
    # For each p_id that has more than one value check if it has more than one record in other data types
    
    for data_type in ['N2', 'N3','REM', 'N1']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
        group_counts = groups.value_counts()
        
        if group_counts[ind] > 1: 
            pass
            
        else:
            single_pids.append(ind)
            
single_pids = list(np.unique(single_pids))

print(single_pids)

['4411', '3305', '1114', '1115', '2201']
['3306', '4404']


#### Load Wake Data , drop relevant p_ids (identified above)

In [97]:
#Load wake data _______________________________________________
paths = joblib.load(folder + 'Wake' + '_paths.pkl')
X = joblib.load('Wake_frequency_data.pkl')

#Load groups and y labels _____________________________________
groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 

#Drop p_ids that are not there in all other data types
X['groups'] = groups 
X['y'] = y
X = X[~X['groups'].isin(p_id_to_drop)]

#Drop p_ids (two) that should only be present once
for val in single_pids:
    duplicates = X[X['groups'].duplicated(keep=False) & (X['groups'] == val)]
    X = X.drop(duplicates.index[0])
# X is what I want now, all features plus additional columns called 'groups' and 'y' 
X_wake = X.copy()

X_wake.columns = [col + '_Wake' for col in X_wake.columns] #For joining with other data types later this is important

#These P_ids have two values
two_ind_vals = list(X_wake['groups_Wake'].value_counts()[ (X_wake['groups_Wake'].value_counts() > 1 ) ].index)
two_ind_vals

#X_wake contains all regional features AND groups + y columns. 
# X_wake has 49 rows 

#Sort the dataframe by p_id and reset the index
X_wake = X_wake.sort_values(by = 'groups_Wake' , ascending = True)
X_wake = X_wake.reset_index(drop = True)

#### Join Non-Wake Data to Wake Data

In [131]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'

#Concatenate all dataframes together to 'Concat_df'
concat_df = X_wake.copy()

for data_type in ['N1','N2','N3','REM']:

    #1. Load the data - put features, groups and y labels into data frame X_df
    X_df = joblib.load( data_type + '_frequency_data.pkl' )
    paths = joblib.load(folder + data_type + '_paths.pkl')
    
    groups = pd.Series(paths['s_p_id'])
    s_class_list = pd.Series(paths['s_class_list'])
    y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 
    
    X_df['groups'] = groups 
    X_df['y'] = y

    #2. Keep only the groups that are also present in Wake data
    X_df_groups_reduced = X_df['groups'][X_df['groups'].isin(list(X_wake['groups_Wake'].values)) ]

    #3. Drop those p_ids that are present more than they are in Wake data. E.g. once in wake data but twice here then drop one
    matching_indices = [] #keep track of the indices to keep
    series = X_df_groups_reduced
    for val in np.unique(X_wake['groups_Wake']):
        if val in two_ind_vals:
            index_of_value = series[series == val].index[0]
            matching_indices.append(index_of_value)
            index_of_value = series[series == val].index[1]
            matching_indices.append(index_of_value)
            
        else:
            index_of_value = series[series == val].index[0]
            matching_indices.append(index_of_value)
        
    if len(matching_indices) == 49:
        pass
    else: 
        print('something went wrong')
        
    X_df = X_df.loc[matching_indices]
    X_df = X_df.sort_values(by = 'groups' , ascending = True)
    X_df = X_df.reset_index(drop = True)
    
    # Append data_type to column name
    X_df.columns = [col + '_' + data_type for col in X_df.columns]
    
    concat_df = pd.concat([concat_df, X_df], axis = 1)

#### Check that Pid's match across data types

In [132]:
data_types = ['Wake', 'N1','N2','N3','REM']
cols_to_select = ['groups_' + data_type for data_type in data_types]
print(concat_df[cols_to_select].nunique(axis = 1).eq(1).all())

cols_to_select = ['y_' + data_type for data_type in data_types]
print(concat_df[cols_to_select].nunique(axis = 1).eq(1).all())

True
True


#### Save data as appropriate

In [133]:
y = concat_df['y_REM']
groups = concat_df['groups_REM']
data_types = ['Wake', 'N1','N2','N3','REM']
cols_to_drop = ['groups_' + data_type for data_type in data_types] + ['y_' + data_type for data_type in data_types]
concat_df = concat_df.drop(columns = cols_to_drop)

In [134]:
joblib.dump(y , 'y_concat.pkl')
joblib.dump(groups, 'groups_concat.pkl')
joblib.dump(concat_df, 'concat_df.pkl')

['concat_df.pkl']